### Segment Fits for ICESat-2 ATL03 data
This notebook uses standard python tools to reduce ICESat-2 ATL03 product to average segments of a specified length

[NSIDC: ICESat-2 Level-2 Products](https://nsidc.org/data/icesat-2/products/level-2)  

The primary and secondary instrumentation onboard the ICESat-2 observatory are the Advanced Topographic Laser Altimeter System (ATLAS, a photon-counting laser altimeter), the global positioning system (GPS) and the star cameras. 
Data from these instruments are combined to create three primary measurements: the time of flight of a photon transmitted and received from ATLAS, the position of the satellite in space, and the pointing vector of the satellite during the transmission of photons. 
These three measurements are used to create ATL03, the geolocated photon product of ICESat-2.  

#### ATL03 - Global Geolocated Photon Data
- Precise latitude, longitude and elevation for every received photon, arranged by beam in the along-track direction  
- Photons classified by signal vs. background, as well as by surface type (land ice, sea ice, land, ocean), including all geophysical corrections  

More information about ATL03 can be found in the Algorithm Theoretical Basis Documents (ATBDs) provided by the ICESat-2 project:  
- [ATL03: Global Geolocated Photon Data](https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/ICESat2_ATL03_ATBD_r002.pdf)  
- [ATL03g: Received photon geolocation](https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/ICESat2_ATL03g_ATBD_r002.pdf)  
- [ATL03a: Atmospheric Delay Corrections](https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/I2_ATL03A_ATBD.pdf)  

#### Load necessary modules for running the notebook

In [ ]:
from __future__ import print_function, division

import os
import re
import h5py
import logging
import datetime
import numpy as np
import scipy.interpolate
import ipywidgets as widgets

import icesat2_toolkit.fit
import icesat2_toolkit.time
import icesat2_toolkit.utilities
from icesat2_toolkit.convert_delta_time import convert_delta_time
from icesat2_toolkit.read_ICESat2_ATL03 import read_HDF5_ATL03, read_HDF5_ATL09
from yapc.classify_photons import classify_photons
logging.basicConfig(level=logging.INFO)

#### Set working data directory and segment fit length

In [ ]:
# set the directory with ICESat-2 data
dirText = widgets.Text(
    value=os.getcwd(),
    description='Directory:',
    disabled=False
)

# slider for setting length of segment in meters
LengthSlider = widgets.IntSlider(
    value=40,
    min=10,
    max=200,
    step=10,
    description='Length:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

# display widgets for setting ICESat-2 parameters
widgets.VBox([dirText,LengthSlider])

#### Select ATL03 and associated ATL09 file

In [ ]:
# set the ATL03 file
ATL03Text = widgets.Text(
    value='ATL03_20191115042423_07520512_004_01.h5',
    description='ATL03 File:',
    disabled=False
)
# set the ATL09 file
ATL09Text = widgets.Text(
    value='ATL09_20191115031051_07520501_004_01.h5',
    description='ATL09 File:',
    disabled=False
)
# display widgets for setting ICESat-2 files
widgets.VBox([ATL03Text,ATL09Text])

#### Download ATL03 and ATL09 HDF5 files from NSIDC

In [ ]:
# set values from widgets
base_dir = dirText.value
ATL03_file = ATL03Text.value
ATL09_file = ATL09Text.value
# regular expression for extracting parameters from ATL03 and ATL09 files
rx = re.compile(r'(processed_)?(ATL\d{2})_(\d{4})(\d{2})(\d{2})(\d{2})'
    '(\d{2})(\d{2})_(\d{4})(\d{2})(\d{2})_(\d{3})_(\d{2})(.*?).h5$')
for base_file in [ATL03_file,ATL09_file]:
    #-- extract parameters from ICESat-2 ATLAS HDF5 file name
    SUB,PRD,YY,MM,DD,HH,MN,SS,TRK,CYCL,GRAN,RL,VERS,AUX = rx.findall(base_file).pop()
    if not os.access(os.path.join(base_dir,base_file),os.F_OK):
        file_url = ['https://n5eil01u.ecs.nsidc.org','ATLAS',
        '{0}.{1}'.format(PRD,RL),'{0}.{1}.{2}'.format(YY,MM,DD),base_file]
        buffer,error = icesat2_toolkit.utilities.from_nsidc(file_url,
            local=os.path.join(base_dir,base_file),verbose=True)
        logging.critical(error) if not buffer else None

#### Read ATL03 HDF5 file and extract variables of interest
The structure of the ATL03 file has (at most) six beam groups, data describing the responses of the ATLAS instrument, ancillary data for correcting and transforming the ATL03 data, and a group of metadata.  


#### Extract photon data and calculate means along overlapping segments

ATL03 contains most of the data needed to create the higher level data products (such as the ATL06 land ice product).  Here, we will calculate the mean elevation of segments for each beam to be used for later visualization.  These mean segment elevations will be corrected for transmit pulse shape biases and first photon biases.

The ATL03 photon events will have a confidence level flag associated with it for a given surface type:  
- -2: possible Transmit Echo Photons  
- -1: events not associated with a specific surface type  
- 0: noise  
- 1: buffer but algorithm classifies as background  
- 2: low  
- 3: medium  
- 4: high  

In the confidence level matrix, the column of each surface types is:   
- 0: Land  
- 1: Ocean  
- 2: Sea Ice  
- 3: Land Ice  
- 4: Inland Water  

The level of confidence of a given photon event (PE) may vary based on the surface type

In [ ]:
# load ICESat-2 ATL03 data
IS2_atl03_mds,IS2_atl03_attrs,IS2_atl03_beams = \
    read_HDF5_ATL03(os.path.join(base_dir,ATL03_file),
    ATTRIBUTES=True)

# universal variables
# speed of light
c = 299792458.0
# associated beam pairs
associated_beam_pair = dict(gt1l='gt1r',gt1r='gt1l',
    gt2l='gt2r',gt2r='gt2l',gt3l='gt3r',gt3r='gt3l')

# copy variables for outputting to HDF5 file
IS2_atl03_fit = {}
IS2_atl03_fit_attrs = {}
IS2_atl03_fill = {}

# value of segment fit length of widget
segment_fit_length = LengthSlider.value
# number of GPS seconds between the GPS epoch
# and ATLAS Standard Data Product (SDP) epoch
atlas_sdp_gps_epoch, = IS2_atl03_mds['ancillary_data']['atlas_sdp_gps_epoch']
# which TEP to use for a given spot (convert to 0-based index)
tep_valid_spot = IS2_atl03_mds['ancillary_data']['tep']['tep_valid_spot'][:] - 1
tep_pce = ['pce1_spot1','pce2_spot3']
# valid range of times for each TEP histogram
tep_range_prim = IS2_atl03_mds['ancillary_data']['tep']['tep_range_prim'][:]
# save tep parameters for a given beam
tep = {}


#### Estimating First Photon Bias (FPB) correction

The ATLAS instrument decides whether or not to telemeter packets of received photons back as data.  ATLAS uses a digital elevation model (DEM) and a few rules to decide whether to transmit large blocks of data to NASA.  The telemetry bands are evident by the spread of low confidence photon events around the surface for each beam.  

Photon events in ATL03 can come to the ATLAS receiver in a few different ways:  
- Many photons come from the sun either by reflecting off clouds or the land surface.  These photon events are spread in a random distribution along the telemetry band.  In ATL03, a large majority of these "background" photon events are classified, but some may be incorrectly classified as signal.  
- Some photons will be returns from the [Transmit Echo Path (TEP)](https://nsidc.org/sites/nsidc.org/files/technical-references/ATL03_Known_Issues_May2019.pdf)  
- Some photons are from the ATLAS instrument that have reflected off clouds. These photons can be clustered together or widely dispersed depending on the properties of the cloud and a few other variables.  
- Some photons are from the ATLAS instrument that have reflected off the surface (our signal photons).  

There will be photons transmitted by the ATLAS instrument will never be recorded back.  The vast majority of these photons never reached the ATLAS instrument again  (only about 10 out of the 10<sup>14</sup> photons transmitted are received), but some are not detected due to the "dead time" of the instrument.  This can create a bias towards the first photons that were received by the instrument.  This first photon bias (FPB) is estimated in the functions below using a model of the gain of the detector.

#### Estimating Transmit Pulse Shape (TPS) correction

The transmitted pulse from ATLAS is also not symmetric in time, which can introduce a bias when calculating average surfaces.  The magnitude of this bias depends on the shape of the transmitted waveform, the width of the window used to calculate the average surface, and the slope and roughness of the surface that broadens the return pulse.  This transmit-pulse shape bias is also estimated using histograms calculated using photons from the Transmit Echo Path (TEP) and an estimate of the width of the histogram of received photons.

#### Calculate average surface height for each beam
- Estimate First Photon Bias (FPB)
- Estimate Transmit Pulse Shape Correction

In [ ]:
# for each input beam within the file
for gtx in sorted(IS2_atl03_beams):
    # data and attributes for beam gtx
    val = IS2_atl03_mds[gtx]
    attrs = IS2_atl03_attrs[gtx]
    # ATLAS spot number for beam in current orientation
    spot = int(attrs['atlas_spot_number'])
    #-- beam type (weak versus strong) for time
    atlas_beam_type = attrs['atlas_beam_type'].decode('utf-8')
    n_pixels = 16.0 if (atlas_beam_type == "strong") else 4.0

    # output dictionaries for variables, attributes and fill values
    IS2_atl03_fit[gtx] = dict(land_ice_segments={})
    IS2_atl03_fit_attrs[gtx] = dict(land_ice_segments={})
    IS2_atl03_fill[gtx] = dict(land_ice_segments={})

    # group attributes for beam
    IS2_atl03_fit_attrs[gtx]['Description'] = attrs['Description']
    IS2_atl03_fit_attrs[gtx]['atlas_pce'] = attrs['atlas_pce']
    IS2_atl03_fit_attrs[gtx]['atlas_beam_type'] = attrs['atlas_beam_type']
    IS2_atl03_fit_attrs[gtx]['groundtrack_id'] = attrs['groundtrack_id']
    IS2_atl03_fit_attrs[gtx]['atmosphere_profile'] = attrs['atmosphere_profile']
    IS2_atl03_fit_attrs[gtx]['atlas_spot_number'] = attrs['atlas_spot_number']
    IS2_atl03_fit_attrs[gtx]['sc_orientation'] = attrs['sc_orientation']
    # group attributes for land_ice_segments
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['Description'] = ("The land_ice_segments group "
        "contains the primary set of derived products. This includes geolocation, height, and "
        "standard error and quality measures for each segment. This group is sparse, meaning "
        "that parameters are provided only for pairs of segments for which at least one beam "
        "has a valid surface-height measurement.")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['data_rate'] = ("Data within this group are "
        "sparse.  Data values are provided only for those ICESat-2 segments where at "
        "least one beam has a valid land ice height measurement.")

    # dem variables
    IS2_atl03_fit[gtx]['land_ice_segments']['dem'] = {}
    IS2_atl03_fill[gtx]['land_ice_segments']['dem'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['dem'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['dem']['Description'] = ("The dem group "
        "contains the reference digital elevation model and geoid heights.")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['dem']['data_rate'] = ("Data within this group "
        "are stored at the land_ice_segments segment rate.")

    # geophysical variables
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical'] = {}
    IS2_atl03_fill[gtx]['land_ice_segments']['geophysical'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['Description'] = ("The geophysical group "
        "contains parameters used to correct segment heights for geophysical effects, parameters "
        "related to solar background and parameters indicative of the presence or absence of clouds.")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['data_rate'] = ("Data within this group "
        "are stored at the land_ice_segments segment rate.")

    # bias correction variables
    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction'] = {}
    IS2_atl03_fill[gtx]['land_ice_segments']['bias_correction'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['Description'] = ("The bias_correction group "
        "contains information about the estimated first-photon bias, and the transmit-pulse-shape bias.")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['data_rate'] = ("Data within this group "
        "are stored at the land_ice_segments segment rate.")

    # fit statistics variables
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics'] = {}
    IS2_atl03_fill[gtx]['land_ice_segments']['fit_statistics'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['Description'] = ("The fit_statistics group "
        "contains a variety of parameters that might indicate the quality of the fitted segment data. Data in "
        "this group are sparse, with dimensions matching the land_ice_segments group.")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['data_rate'] = ("Data within this group "
        "are stored at the land_ice_segments segment rate.")

    # ground track variables
    IS2_atl03_fit[gtx]['land_ice_segments']['ground_track'] = {}
    IS2_atl03_fill[gtx]['land_ice_segments']['ground_track'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['Description'] = ("The ground_track group "
        "contains parameters describing the GT and RGT for each land ice segment, as well as angular "
        "information about the beams.")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['data_rate'] = ("Data within this group "
        "are stored at the land_ice_segments segment rate.")
    
    # ATL03 Segment ID
    ATL03_Segment_ID = val['geolocation']['segment_id']
    # first photon in the segment (convert to 0-based indexing)
    Segment_Index_begin = val['geolocation']['ph_index_beg'] - 1
    # number of photon events in the segment
    Segment_PE_count = val['geolocation']['segment_ph_cnt']
    # along-track distance for each ATL03 segment
    Segment_Distance = val['geolocation']['segment_dist_x']
    # along-track length for each ATL03 segment
    Segment_Length = val['geolocation']['segment_length']
    # output fill value for floating point variables
    fill_value = attrs['geolocation']['sigma_h']['_FillValue']

    # number of valid overlapping output segments
    total_distance = Segment_Distance[-1] - Segment_Distance[0] + Segment_Length[-1]
    n_seg = int(total_distance/(0.5*segment_fit_length))

    # get ATLAS impulse response variables for the transmitter echo path (TEP)
    tep1,tep2 = ('atlas_impulse_response','tep_histogram')
    # get appropriate transmitter-echo-path histogram for spot
    associated_pce = tep_valid_spot[spot-1]
    pce = tep_pce[associated_pce]
    # delta time of TEP histogram
    tep_tod, = IS2_atl03_mds[tep1][pce][tep2]['tep_tod'][:]
    # truncate tep to primary histogram (reflection 43-50 ns)
    # and extract signal tep from noise tep.  calculate width of tep
    # ATL03 recommends subsetting between 15-30 ns to avoid secondary
    tep_hist_time = np.copy(IS2_atl03_mds[tep1][pce][tep2]['tep_hist_time'][:])
    tep_hist = np.copy(IS2_atl03_mds[tep1][pce][tep2]['tep_hist'][:])    
    t_TX,p_TX,W_TX,FWHM,TXs,TXe = icesat2_toolkit.fit.extract_tep_histogram(
        tep_hist_time, tep_hist, tep_range_prim)
    # save tep information and statistics for HDF5 file attributes
    tep[gtx] = {}
    tep[gtx]['pce'] = pce
    tep[gtx]['tep_tod'] = tep_tod
    tep[gtx]['tx_start'] = TXs
    tep[gtx]['tx_end'] = TXe
    tep[gtx]['tx_robust_sprd'] = W_TX
    tep[gtx]['sigma_tx'] = FWHM

    # channel dead time and first photon bias table for beam
    cal1,cal2 = ('ancillary_data','calibrations')
    channel_dead_time = IS2_atl03_mds[cal1]['dead_time'][gtx]['dead_time'][:]
    mean_dead_time = np.mean(channel_dead_time)
    fpb_dead_time = IS2_atl03_mds[cal1]['first_photon_bias'][gtx]['dead_time'][:]
    fpb_strength = IS2_atl03_mds[cal1]['first_photon_bias'][gtx]['strength'][:]
    fpb_width = IS2_atl03_mds[cal1]['first_photon_bias'][gtx]['width'][:]
    fpb_corr = IS2_atl03_mds[cal1]['first_photon_bias'][gtx]['ffb_corr'][:]
    # calculate first photon bias as a function of strength and width
    # for the calculated mean dead time of the beam
    ndt,ns,nw = np.shape(fpb_corr)
    fpb_corr_dead_time = np.zeros((ns,nw))
    for s in range(ns):
        for w in range(nw):
            SPL = scipy.interpolate.UnivariateSpline(fpb_dead_time/1e9,
                fpb_corr[:,s,w],k=3,s=0)
            fpb_corr_dead_time[s,w] = SPL(mean_dead_time)
    # bivariate spline for estimating first-photon bias using CAL-19
    CAL19 = scipy.interpolate.RectBivariateSpline(fpb_strength[0,:],
        fpb_width[0,:]/1e9, fpb_corr_dead_time/1e12, kx=1, ky=1)

    # allocate for output variables
    # land ice segment group variables
    # start, end and geolocated mean segment id
    # first ATL03 segment ID used in output segment
    IS2_atl03_fit[gtx]['land_ice_segments']['segment_id_start'] = np.full((n_seg),-1,dtype=int)
    IS2_atl03_fill[gtx]['land_ice_segments']['segment_id_start'] = -1
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['segment_id_start'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['segment_id_start']['units'] = "1"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['segment_id_start']['contentType'] = "referenceInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['segment_id_start']['long_name'] = "Along-track segment ID number"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['segment_id_start']['description'] = ("A 7 digit number "
        "identifying the along-track geolocation segment number.  These are sequential, starting with "
        "1 for the first segment after an ascending equatorial crossing node. Equal to the segment_id for "
        "the second of the two 20m ATL03 segments included in the 40m ATL06 segment")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['segment_id_start']['coordinates'] = \
        "delta_time latitude longitude"  
    
    # final ATL03 segment ID used in output segment
    IS2_atl03_fit[gtx]['land_ice_segments']['segment_id_end'] = np.full((n_seg),-1,dtype=int)
    IS2_atl03_fill[gtx]['land_ice_segments']['segment_id_end'] = -1
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['segment_id_end'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['segment_id_end']['units'] = "1"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['segment_id_end']['contentType'] = "referenceInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['segment_id_end']['long_name'] = "Along-track segment ID number"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['segment_id_end']['description'] = ("A 7 digit number "
        "identifying the along-track geolocation segment number.  These are sequential, starting with "
        "1 for the first segment after an ascending equatorial crossing node. Equal to the segment_id for "
        "the second of the two 20m ATL03 segments included in the 40m ATL06 segment")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['segment_id_end']['coordinates'] = \
        "delta_time latitude longitude"
    
    # ATL03 segment ID for output segment center
    IS2_atl03_fit[gtx]['land_ice_segments']['segment_id'] = np.full((n_seg),-1,dtype=int)
    IS2_atl03_fill[gtx]['land_ice_segments']['segment_id'] = -1
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['segment_id'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['segment_id']['units'] = "1"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['segment_id']['contentType'] = "referenceInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['segment_id']['long_name'] = "Along-track segment ID number"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['segment_id']['description'] = ("A 7 digit number "
        "identifying the along-track geolocation segment number.  These are sequential, starting with "
        "1 for the first segment after an ascending equatorial crossing node. Equal to the segment_id for "
        "the second of the two 20m ATL03 segments included in the 40m ATL06 segment")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['segment_id']['coordinates'] = \
        "delta_time latitude longitude"
    
    # delta time of fit photons
    IS2_atl03_fit[gtx]['land_ice_segments']['delta_time'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['delta_time'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['delta_time'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['delta_time'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['delta_time']['units'] = "seconds since 2018-01-01"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['delta_time']['long_name'] = "Elapsed GPS seconds"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['delta_time']['standard_name'] = "time"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['delta_time']['calendar'] = "standard"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['delta_time']['description'] = ("Number of GPS "
        "seconds since the ATLAS SDP epoch. The ATLAS Standard Data Products (SDP) epoch offset "
        "is defined within /ancillary_data/atlas_sdp_gps_epoch as the number of GPS seconds "
        "between the GPS epoch (1980-01-06T00:00:00.000000Z UTC) and the ATLAS SDP epoch. By "
        "adding the offset contained within atlas_sdp_gps_epoch to delta time parameters, the "
        "time in gps_seconds relative to the GPS epoch can be computed.")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['delta_time']['coordinates'] = \
        "segment_id latitude longitude"
    
    # land ice height corrected for first photon bias and transmit-pulse shape
    IS2_atl03_fit[gtx]['land_ice_segments']['h_li'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['h_li'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['h_li'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['h_li'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['h_li']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['h_li']['contentType'] = "physicalMeasurement"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['h_li']['long_name'] = "Land Ice height"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['h_li']['description'] = ("Standard land-ice segment "
        "height determined by land ice algorithm, corrected for first-photon bias, representing the "
        "median-based height of the selected PEs")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['h_li']['coordinates'] = \
        "segment_id delta_time latitude longitude"
    
    # land ice height errors (max of fit or first photon bias uncertainties)
    IS2_atl03_fit[gtx]['land_ice_segments']['h_li_sigma'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['h_li_sigma'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['h_li_sigma'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['h_li_sigma'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['h_li_sigma']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['h_li_sigma']['contentType'] = "qualityInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['h_li_sigma']['long_name'] = "Expected RMS segment misfit"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['h_li_sigma']['description'] = ("Propagated error due to "
        "sampling error and FPB correction from the land ice algorithm")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['h_li_sigma']['coordinates'] = \
        "segment_id delta_time latitude longitude"
    
    # longitude of fit photons
    IS2_atl03_fit[gtx]['land_ice_segments']['longitude'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['longitude'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['longitude'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['longitude'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['longitude']['units'] = "degrees_east"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['longitude']['contentType'] = "physicalMeasurement"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['longitude']['long_name'] = "Longitude"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['longitude']['standard_name'] = "longitude"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['longitude']['description'] = ("Longitude of "
        "segment center")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['longitude']['valid_min'] = -180.0
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['longitude']['valid_max'] = 180.0
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['longitude']['coordinates'] = \
        "segment_id delta_time latitude"
    
    # latitude of fit photons
    IS2_atl03_fit[gtx]['land_ice_segments']['latitude'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['latitude'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['latitude'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['latitude'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['latitude']['units'] = "degrees_north"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['latitude']['contentType'] = "physicalMeasurement"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['latitude']['long_name'] = "Latitude"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['latitude']['standard_name'] = "latitude"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['latitude']['description'] = ("Latitude of "
        "segment center")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['latitude']['valid_min'] = -90.0
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['latitude']['valid_max'] = 90.0
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['latitude']['coordinates'] = \
        "segment_id delta_time longitude"
    
    # segment quality summary
    IS2_atl03_fit[gtx]['land_ice_segments']['atl06_quality_summary'] = np.ma.zeros((n_seg),fill_value=-1,dtype=int)
    IS2_atl03_fit[gtx]['land_ice_segments']['atl06_quality_summary'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['atl06_quality_summary'] = -1
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['atl06_quality_summary'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['atl06_quality_summary']['units'] = "1"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['atl06_quality_summary']['contentType'] = "qualityInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['atl06_quality_summary']['long_name'] = "ATL06 Quality Summary"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['atl06_quality_summary']['description'] = ("The ATL06_quality_summary "
        "parameter indicates the best-quality subset of all ATL06 data. A zero in this parameter implies that no "
        "data-quality tests have found a problem with the segment, a one implies that some potential problem has "
        "been found. Users who select only segments with zero values for this flag can be relatively certain of "
        "obtaining high-quality data, but will likely miss a significant fraction of usable data, particularly in "
        "cloudy, rough, or low-surface-reflectance conditions.")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['atl06_quality_summary']['flag_meanings'] = \
        "best_quality potential_problem"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['longitude']['valid_min'] = 0
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['longitude']['valid_max'] = 1
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['atl06_quality_summary']['coordinates'] = \
        "segment_id delta_time latitude longitude"

    # geophysical corrections and variables
    # background rate
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['bckgrd'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['bckgrd'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['geophysical']['bckgrd'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bckgrd'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bckgrd']['units'] = "counts / second"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bckgrd']['contentType'] = "modelResult"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bckgrd']['long_name'] = ("Background count "
        "rate based on the ATLAS 50-shot sum interpolated to the reference photon")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bckgrd']['description'] = ("The background "
        "count rate from the 50-shot altimetric histogram after removing the number of likely signal photons")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bckgrd']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"

    # bias corrections
    # difference between the mean and median of fit residuals
    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['med_r_fit'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['med_r_fit'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['bias_correction']['med_r_fit'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['med_r_fit'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['med_r_fit']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['med_r_fit']['contentType'] = "qualityInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['med_r_fit']['long_name'] = "mean median residual"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['med_r_fit']['description'] = ("Difference between "
        "uncorrected mean and median of linear fit residuals")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['med_r_fit']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    
    # first photon bias estimates    
    # mean first photon bias
    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_mean_corr'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_mean_corr'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['bias_correction']['fpb_mean_corr'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_mean_corr'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_mean_corr']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_mean_corr']['contentType'] = "qualityInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_mean_corr']['long_name'] = "first photon bias mean correction"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_mean_corr']['description'] = ("Estimated first-photon-bias "
        "(fpb) correction to mean segment height")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_mean_corr']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    
    # mean first photon bias uncertainty
    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_mean_corr_sigma'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_mean_corr_sigma'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['bias_correction']['fpb_mean_corr_sigma'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_mean_corr_sigma'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_mean_corr_sigma']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_mean_corr_sigma']['contentType'] = "qualityInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_mean_corr_sigma']['long_name'] = ("first photon bias "
        "mean correction error")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_mean_corr_sigma']['description'] = ("Estimated error in "
        "first-photon-bias (fpb) correction for mean segment heights")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_mean_corr_sigma']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    
    # median first photon bias
    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr']['contentType'] = "qualityInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr']['long_name'] = "first photon bias median correction"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr']['description'] = ("Estimated first-photon-bias "
        "(fpb) correction giving the difference between the mean segment height and the corrected median height")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    
    # median first photon bias correction
    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr_sigma'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr_sigma'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr_sigma'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr_sigma'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr_sigma']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr_sigma']['contentType'] = "qualityInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr_sigma']['long_name'] = ("first photon bias median "
        "correction error")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr_sigma']['description'] = ("Estimated error in "
        "first-photon-bias (fpb) correction giving the difference between the mean segment height and the corrected median height")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr_sigma']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    
    # first photon bias corrected number of photons
    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_n_corr'] = np.ma.zeros((n_seg),fill_value=-1,dtype=int)
    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_n_corr'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['bias_correction']['fpb_n_corr'] = -1
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_n_corr'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_n_corr']['units'] = "1"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_n_corr']['contentType'] = "qualityInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_n_corr']['long_name'] = "corrected number of photons"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_n_corr']['description'] = ("Estimated photon count after "
        "first-photon-bias correction")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_n_corr']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    
    # CAL-19 first photon bias
    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_cal_corr'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_cal_corr'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['bias_correction']['fpb_cal_corr'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_cal_corr'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_cal_corr']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_cal_corr']['contentType'] = "qualityInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_cal_corr']['long_name'] = "first photon bias calibrated correction"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_cal_corr']['description'] = ("Estimated first-photon-bias "
        "(fpb) correction calculated using the ATL03 calibration products")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['fpb_cal_corr']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    
    # transmit pulse shape bias estimates
    # mean transmit pulse shape correction
    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['tx_mean_corr'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['tx_mean_corr'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['bias_correction']['tx_mean_corr'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['tx_mean_corr'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['tx_mean_corr']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['tx_mean_corr']['contentType'] = "qualityInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['tx_mean_corr']['long_name'] = "tx shape mean correction"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['tx_mean_corr']['description'] = ("Estimate of the difference "
        "between the mean of the full-waveform transmit-pulse and the mean of a broadened, truncated waveform consistent "
        "with the received pulse")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['tx_mean_corr']['source'] = tep[gtx]['pce']
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['tx_mean_corr']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    
    # median transmit pulse shape correction
    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['tx_med_corr'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['tx_med_corr'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['bias_correction']['tx_med_corr'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['tx_med_corr'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['tx_med_corr']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['tx_med_corr']['contentType'] = "qualityInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['tx_med_corr']['long_name'] = "tx shape median correction"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['tx_med_corr']['description'] = ("Estimate of the difference "
        "between the median of the full-waveform transmit-pulse and the median of a broadened, truncated waveform consistent "
        "with the received pulse")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['tx_med_corr']['source'] = tep[gtx]['pce']
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['bias_correction']['tx_med_corr']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    
    # segment fit statistics
    # segment fit heights
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['h_mean'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['h_mean'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['fit_statistics']['h_mean'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['h_mean'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['h_mean']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['h_mean']['contentType'] = "modelResult"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['h_mean']['long_name'] = "Height Mean"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['h_mean']['description'] = ("Mean surface "
        "height, not corrected for first-photon bias or pulse truncation")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['h_mean']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    
    # segment fit along-track slopes
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dx'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dx'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dx'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dx'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dx']['units'] = "meters/meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dx']['contentType'] = "modelResult"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dx']['long_name'] = "Along Track Slope"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dx']['description'] = ("Along-track slope "
        "from along-track segment fit")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dx']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    
    # segment fit height errors
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['sigma_h_mean'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['sigma_h_mean'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['fit_statistics']['sigma_h_mean'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['sigma_h_mean'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['sigma_h_mean']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['sigma_h_mean']['contentType'] = "qualityInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['sigma_h_mean']['long_name'] = "Height Error"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['sigma_h_mean']['description'] = ("Propagated height "
        "error due to PE-height sampling error for height from the along-track fit, not including geolocation-induced error")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['sigma_h_mean']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    
    # segment fit across-track slope errors
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dx_sigma'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dx_sigma'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dx_sigma'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dx_sigma'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dx_sigma']['units'] = "meters/meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dx_sigma']['contentType'] = "qualityInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dx_sigma']['long_name'] = "Sigma of Along Track Slope"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dx_sigma']['description'] = ("Propagated error in "
        "the along-track segment slope")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dx_sigma']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    
    # number of photons in fit
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['n_fit_photons'] = np.ma.zeros((n_seg),fill_value=-1,dtype=int)    
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['n_fit_photons'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['fit_statistics']['n_fit_photons'] = -1
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['n_fit_photons'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['n_fit_photons']['units'] = "1"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['n_fit_photons']['contentType'] = "physicalMeasurement"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['n_fit_photons']['long_name'] = "Number of Photons in Fit"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['n_fit_photons']['description'] = ("Number of PEs used to "
        "determine mean surface height in the iterative surface fit")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['n_fit_photons']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    
    # size of the window used in the fit
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['w_surface_window_final'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['w_surface_window_final'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['fit_statistics']['w_surface_window_final'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['w_surface_window_final'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['w_surface_window_final']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['w_surface_window_final']['contentType'] = "physicalMeasurement"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['w_surface_window_final']['long_name'] = "Surface Window Width"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['w_surface_window_final']['description'] = ("Width of the surface "
        "window, top to bottom")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['w_surface_window_final']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    
    # signal-to-noise ratio
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['snr'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['snr'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['fit_statistics']['snr'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['snr'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['snr']['units'] = "1"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['snr']['contentType'] = "physicalMeasurement"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['snr']['long_name'] = "SNR"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['snr']['description'] = ("Signal-to-noise "
        "ratio in the final refined window")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['snr']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    
    # robust dispersion estimator
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['h_robust_sprd'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['h_robust_sprd'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['fit_statistics']['h_robust_sprd'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['h_robust_sprd'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['h_robust_sprd']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['h_robust_sprd']['contentType'] = "qualityInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['h_robust_sprd']['long_name'] = "Robust Spread"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['h_robust_sprd']['description'] = ("RDE of misfit "
        "between PE heights and the along-track segment fit")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['h_robust_sprd']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    
    # number of iterations for fit
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['n_fit_iterations'] = np.ma.zeros((n_seg),fill_value=-1,dtype=int)
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['n_fit_iterations'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['fit_statistics']['n_fit_iterations'] = -1
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['n_fit_iterations'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['n_fit_iterations']['units'] = "1"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['n_fit_iterations']['contentType'] = "modelResult"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['n_fit_iterations']['long_name'] = "Number of Iterations used in Fit"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['n_fit_iterations']['description'] = ("Number of Iterations when "
        "determining the mean surface height")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['n_fit_iterations']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    
    # signal source selection
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['signal_selection_source'] = np.ma.zeros((n_seg),fill_value=4,dtype=int)
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['signal_selection_source'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['fit_statistics']['signal_selection_source'] = 4
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['signal_selection_source'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['signal_selection_source']['units'] = "1"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['signal_selection_source']['contentType'] = "qualityInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['signal_selection_source']['long_name'] = "Signal Selection Source"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['signal_selection_source']['description'] = ("Indicates the last "
        "algorithm attempted to select the signal for fitting. 1=Signal selection succeeded using ATL03 detected PE; 2=Signal "
        "selection failed using ATL03 detected PE but succeeded using all flagged ATL03 PE; 3=Signal selection failed using "
        "all flagged ATL03 PE, but succeeded using the backup algorithm; 4=All signal-finding strategies failed.")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['signal_selection_source']['flag_values'] = [1,2,3,4]
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['signal_selection_source']['valid_min'] = 1
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['signal_selection_source']['valid_max'] = 4
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['signal_selection_source']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    
    # number of pulses included in segment
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['n_seg_pulses'] = np.ma.zeros((n_seg),fill_value=-1,dtype=int)
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['n_seg_pulses'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['fit_statistics']['n_seg_pulses'] = -1
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['n_seg_pulses'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['n_seg_pulses']['units'] = "1"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['n_seg_pulses']['contentType'] = "referenceInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['n_seg_pulses']['long_name'] = "Number potential segment pulses"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['n_seg_pulses']['description'] = ("The number of pulses "
        "potentially included in the segment")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['n_seg_pulses']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"

    # ground track coordinates
    # along-track X coordinates of segment fit
    IS2_atl03_fit[gtx]['land_ice_segments']['ground_track']['x_atc'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['ground_track']['x_atc'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['ground_track']['x_atc'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['x_atc'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['x_atc']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['x_atc']['contentType'] = "referenceInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['x_atc']['long_name'] = "X Along Track"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['x_atc']['description'] = ("The along-track "
        "x-coordinate of the segment, measured parallel to the RGT, measured from the ascending node of the equatorial "
        "crossing of a given RGT.")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['x_atc']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    
    # along-track Y coordinates of segment fit
    IS2_atl03_fit[gtx]['land_ice_segments']['ground_track']['y_atc'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['ground_track']['y_atc'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['ground_track']['y_atc'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['y_atc'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['y_atc']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['y_atc']['contentType'] = "referenceInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['y_atc']['long_name'] = "Y Along Track"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['y_atc']['description'] = ("The along-track "
        "y-coordinate of the segment, relative to the RGT, measured along the perpendicular to the RGT, "
        "positive to the right of the RGT.")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['y_atc']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    
    # along-track X coordinate spread of points used in segment fit
    IS2_atl03_fit[gtx]['land_ice_segments']['ground_track']['x_spread'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['ground_track']['x_spread'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['ground_track']['x_spread'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['x_spread'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['x_spread']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['x_spread']['contentType'] = "referenceInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['x_spread']['long_name'] = "X Along Track Spread"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['x_spread']['description'] = ("The spread of "
        "along-track x-coordinates for points used in the segment fit.  Coordinates measured parallel to the "
        "RGT, measured from the ascending node of the equatorial crossing of a given RGT.")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['x_spread']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"

    # iterate over ATL03 segments to along track distances
    n_pe = len(val['heights']['delta_time'])
    photon_event_id = np.zeros((n_pe))
    # along-track X and Y coordinates
    x_atc = np.ma.array(val['heights']['dist_ph_along'],
        mask=np.ones_like(val['heights']['dist_ph_along'],dtype=bool))
    y_atc = np.copy(val['heights']['dist_ph_across'])
    # photon event heights
    h_ph = np.copy(val['heights']['h_ph'])
    #-- digital elevation model interpolated to photon events
    dem_h = np.zeros((n_pe))
    # in ATL03 1-based indexing: invalid == 0
    # here in 0-based indexing: invalid == -1   
    segment_indices, = np.nonzero(Segment_Index_begin >= 0)
    for j in segment_indices:
        # index for ATL03 segment j
        idx = Segment_Index_begin[j]
        # number of photons in ATL03 segment
        cnt = np.copy(Segment_PE_count[j])
        # ATL03 segment ID for each photon event
        photon_event_id[idx:idx+cnt] = ATL03_Segment_ID[j]
        # total along-track distance for each photon event
        # add segment distance to along-track coordinates
        x_atc.data[idx:idx+cnt] += Segment_Distance[j]
        x_atc.mask[idx:idx+cnt] = False
        #-- interpolate digital elevation model to photon events
        dem_h[idx:idx+cnt] = val['geophys_corr']['dem_h'][j]

    #-- iterate over ATLAS major frames
    photon_mframes = val['heights']['pce_mframe_cnt']
    #-- background ATLAS group variables are based upon 50-shot summations
    #-- PCE Major Frames are based upon 200-shot summations
    pce_mframe_cnt = val['bckgrd_atlas']['pce_mframe_cnt']
    #-- find unique major frames and their indices within background ATLAS group
    #-- (there will 4 background ATLAS time steps for nearly every major frame)
    unique_major_frames,unique_index = np.unique(pce_mframe_cnt,return_index=True)
    #-- number of unique major frames in granule for beam
    major_frame_count = len(unique_major_frames)
    #-- height of each telemetry band for a major frame
    tlm_height = {}
    tlm_height['band1'] = val['bckgrd_atlas']['tlm_height_band1']
    tlm_height['band2'] = val['bckgrd_atlas']['tlm_height_band2']
    #-- elevation above ellipsoid of each telemetry band for a major frame
    tlm_top = {}
    tlm_top['band1'] = val['bckgrd_atlas']['tlm_top_band1']
    tlm_top['band2'] = val['bckgrd_atlas']['tlm_top_band2']
    #-- buffer to telemetry band to set as valid
    tlm_buffer = 100.0
    #-- flag denoting photon events as possible TEP
    if (int(RL) < 4):
        isTEP = np.any((val['heights']['signal_conf_ph'][:]==-2),axis=1)
    else:
        isTEP = (val['heights']['quality_ph'][:] == 3)
    #-- photon event weights and signal-to-noise ratio
    pe_weights = np.zeros((n_pe),dtype=np.float64)
    #-- run for each major frame
    for iteration,idx in enumerate(unique_index):
        #-- sum of telemetry band widths for major frame
        h_win_width = 0.0
        #-- check that each telemetry band is close to DEM
        for b in ['band1','band2']:
            #-- bottom of the telemetry band for major frame
            tlm_bot_band = tlm_top[b][idx] - tlm_height[b][idx]
            if np.any((dem_h[i1[i2]] >= (tlm_bot_band-tlm_buffer)) &
                (dem_h[i1[i2]] <= (tlm_top[b][idx]+tlm_buffer))):
                #-- add telemetry height to window width
                h_win_width += tlm_height[b][idx]
        #-- photon indices for major frame (buffered by 1 frame on each side)
        #-- do not use possible TEP photons in photon classification
        i1, = np.nonzero((photon_mframes >= unique_major_frames[iteration]-1) &
            (photon_mframes <= unique_major_frames[iteration]+1) &
            np.logical_not(isTEP))
        #-- indices for the major frame within the buffered window
        i2, = np.nonzero(photon_mframes[i1] == unique_major_frames[iteration])
        #-- calculate photon event weights
        pe_weights[i1[i2]] = classify_photons(x_atc[i1], h_ph[i1],
            h_win_width, i2, K=3, min_ph=3, min_xspread=1.0,
            min_hspread=0.01, aspect=3, method='linear')

    #-- photon event weights scaled to a single byte
    weight_ph = np.array(255*pe_weights,dtype=np.uint8)
    #-- verify photon event weights
    np.clip(weight_ph, 0, 255, out=weight_ph)

    # interpolate background photon rate based on 50-shot summation
    background_delta_time = val['bckgrd_atlas']['delta_time']
    SPL = scipy.interpolate.UnivariateSpline(background_delta_time,
        val['bckgrd_atlas']['bckgrd_rate'],k=3,s=0)
    background_rate = SPL(val['heights']['delta_time'])
    
    # find segments with photon events
    segment_pe_flag = np.zeros((n_seg),dtype=bool)
    for j in range(n_seg):
        distance_start = Segment_Distance[0] + j*segment_fit_length
        distance_end = Segment_Distance[0] + (j+1)*segment_fit_length
        segment_pe_flag[j] = np.any((x_atc >= distance_start) & 
            (x_atc < distance_end))
    
    # for each segment with photon events
    valid_segments, = np.nonzero(segment_pe_flag)
    for j in valid_segments:
        distance_start = Segment_Distance[0] + j*segment_fit_length
        distance_end = Segment_Distance[0] + (j+1)*segment_fit_length
        idx, = np.nonzero((x_atc >= distance_start) & 
            (x_atc < distance_end))
        # segment photon event ids
        segment_photon_event_IDs = photon_event_id[idx]
        # time of each Photon event (PE)
        segment_times = np.copy(val['heights']['delta_time'][idx])
        # Photon event lat/lon and elevation
        segment_heights = np.copy(h_ph[idx])
        segment_lats = np.copy(val['heights']['lat_ph'][idx])
        segment_lons = np.copy(val['heights']['lon_ph'][idx])
        # Photon event channel and identification
        ID_channel = np.copy(val['heights']['ph_id_channel'][idx])
        ID_pulse = np.copy(val['heights']['ph_id_pulse'][idx])
        n_pulses = np.unique(ID_pulse).__len__()
        frame_number = np.copy(val['heights']['pce_mframe_cnt'][idx])
        # along-track X and Y coordinates
        distance_along_X = np.copy(x_atc[idx:idx+cnt])
        distance_along_Y = np.copy(y_atc[idx:idx+cnt])
        # average background photon rate for segment
        IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['bckgrd'] = np.mean(background_rate[idx])
        IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['bckgrd'].mask[j] = False
        # vertical noise-photon density
        background_density = 2.0*n_pulses*np.mean(background_rate[idx])/c
        # check the spread of photons along-track (must be > 0.5*length)
        along_X_spread = distance_along_X[idx].max() - distance_along_X[idx].min()
        # check confidence level associated with each photon event
        # -2: TEP
        # -1: Events not associated with a specific surface type
        #  0: noise
        #  1: buffer but algorithm classifies as background
        #  2: low
        #  3: medium
        #  4: high
        # Surface types for signal classification confidence
        # 0=Land; 1=Ocean; 2=SeaIce; 3=LandIce; 4=InlandWater
        ice_sig_conf = np.copy(val['heights']['signal_conf_ph'][idx,3])
        ice_sig_low_count = np.count_nonzero(ice_sig_conf > 1)
        #-- indices of TEP classified photons
        ice_sig_tep_pe, = np.nonzero(ice_sig_conf == -2)
        #-- photon event weights from photon classifier
        segment_weights = weight_ph[idx]
        snr_norm = np.max(segment_weights)
        #-- photon event signal-to-noise ratio from photon classifier
        photon_snr = np.zeros((cnt),dtype=int)
        if (snr_norm > 0):
            photon_snr[:] = 100.0*segment_weights/snr_norm
        #-- photon confidence levels from classifier
        pe_sig_conf = np.zeros((cnt),dtype=int)
        #-- calculate confidence levels from photon classifier
        pe_sig_conf[photon_snr >= 25] = 2
        pe_sig_conf[photon_snr >= 60] = 3
        pe_sig_conf[photon_snr >= 80] = 4
        #-- copy classification for TEP photons
        pe_sig_conf[ice_sig_tep_pe] = -2
        pe_sig_low_count = np.count_nonzero(pe_sig_conf > 1)
        # check if segment has photon events classified for land ice
        # that are at or above low-confidence threshold
        # and that the spread of photons is greater than 20m
        if (pe_sig_low_count > 10) & (along_X_spread > 0.5*segment_fit_length):
            # perform a surface fit procedure
            Segment_X = (distance_start + distance_end)/2.0
            valid,fit,centroid = icesat2_toolkit.fit.try_surface_fit(
                distance_along_X, distance_along_Y, segment_heights,
                pe_sig_conf, Segment_X, SURF_TYPE='linear', ITERATE=20,
                CONFIDENCE=[1,0])
            # indices of points used in final iterated fit
            ifit = np.sort(fit['indices']) if valid else None
            if bool(valid) & (np.abs(fit['error'][0]) < 20):
                # average height of segment
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['h_mean'].data[j] = fit['beta'][0]
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['h_mean'].mask[j] = False
                # along track slope
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dx'].data[j] = fit['beta'][1]
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dx'].mask[j] = False
                # height uncertainty of segment
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['sigma_h_mean'].data[j] = fit['error'][0]
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['sigma_h_mean'].mask[j] = False
                # along track slope uncertainty
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dx_sigma'].data[j] = fit['error'][1]
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dx_sigma'].mask[j] = False
                # along-track and cross-track coordinates
                IS2_atl03_fit[gtx]['land_ice_segments']['ground_track']['x_atc'].data[j] = np.copy(centroid['x'])
                IS2_atl03_fit[gtx]['land_ice_segments']['ground_track']['x_atc'].mask[j] = False
                IS2_atl03_fit[gtx]['land_ice_segments']['ground_track']['x_spread'].data[j] = np.copy(along_X_spread)
                IS2_atl03_fit[gtx]['land_ice_segments']['ground_track']['x_spread'].mask[j] = False
                IS2_atl03_fit[gtx]['land_ice_segments']['ground_track']['y_atc'].data[j] = np.copy(centroid['y'])
                IS2_atl03_fit[gtx]['land_ice_segments']['ground_track']['y_atc'].mask[j] = False
                # start and end segment ID of photons used in fit
                pe_start,pe_end = (ifit[0],ifit[-1])
                IS2_atl03_fit[gtx]['land_ice_segments']['segment_id_start'][j] = segment_photon_event_IDs[pe_start]
                IS2_atl03_fit[gtx]['land_ice_segments']['segment_id_end'][j] = segment_photon_event_IDs[pe_end]
                # fit geolocation to the along-track distance of segment
                IS2_atl03_fit[gtx]['land_ice_segments']['segment_id'][j] = \
                    icesat2_toolkit.fit.fit_geolocation(segment_photon_event_IDs[ifit],
                    distance_along_X[ifit], Segment_X_atc[j]).astype(int)
                IS2_atl03_fit[gtx]['land_ice_segments']['delta_time'].data[j] = \
                    icesat2_toolkit.fit.fit_geolocation(segment_times[ifit],
                    distance_along_X[ifit], Segment_X_atc[j])
                IS2_atl03_fit[gtx]['land_ice_segments']['delta_time'].mask[j] = False
                IS2_atl03_fit[gtx]['land_ice_segments']['longitude'].data[j] = \
                    icesat2_toolkit.fit.fit_geolocation(segment_lons[ifit],
                    distance_along_X[ifit], Segment_X_atc[j])
                IS2_atl03_fit[gtx]['land_ice_segments']['longitude'].mask[j] = False
                IS2_atl03_fit[gtx]['land_ice_segments']['latitude'].data[j] = \
                    icesat2_toolkit.fit.fit_geolocation(segment_lats[ifit],
                    distance_along_X[ifit], Segment_X_atc[j])
                IS2_atl03_fit[gtx]['land_ice_segments']['latitude'].mask[j] = False
                
                # number of photons used in fit
                n_fit_photons = len(ifit)
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['n_fit_photons'].data[j] = n_fit_photons
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['n_fit_photons'].mask[j] = False
                # size of the final window
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['w_surface_window_final'].data[j] = np.copy(fit['window'])
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['w_surface_window_final'].mask[j] = False
                # robust dispersion estimator
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['h_robust_sprd'].data[j] = np.copy(fit['RDE'])
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['h_robust_sprd'].mask[j] = False
                # signal to noise ratio
                N_BG = background_density*fit['window']
                snr = n_fit_photons/N_BG
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['snr'].data[j] = np.copy(snr)
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['snr'].mask[j] = False
                # number of iterations used in fit
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['n_fit_iterations'].data[j] = np.copy(fit['iterations'])
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['n_fit_iterations'].mask[j] = False
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['signal_selection_source'].data[j] = np.copy(valid)
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['signal_selection_source'].mask[j] = False
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['n_seg_pulses'].data[j] = np.copy(n_pulses)
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['n_seg_pulses'].mask[j] = False
                # calculate residuals off of fit surface for all data
                x_slope = fit['beta'][1]*(distance_along_X-centroid['x'])
                height_residuals = segment_heights-fit['beta'][0]-x_slope
                temporal_residuals = -2.0*height_residuals/c
                # calculate difference between the mean and the median from the fit
                IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['med_r_fit'].data[j] = \
                    np.mean(height_residuals[ifit]) - np.median(height_residuals[ifit])
                IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['med_r_fit'].mask[j] = False
                # calculate flags for quality summary
                VPD = n_fit_photons/fit['window']
                IS2_atl03_fit[gtx]['land_ice_segments']['atl06_quality_summary'].data[j] = int(
                    (fit['RDE'] >= 1) |
                    (fit['error'][0] >= 1) |
                    (VPD <= (n_pixels/4.0)))
                IS2_atl03_fit[gtx]['land_ice_segments']['atl06_quality_summary'].mask[j] = False

                # estimate first photon bias corrections
                # step-size for histograms (50 ps ~ 7.5mm height)
                try:
                    ii, = np.nonzero((height_residuals >= -fit['window']) &
                        (height_residuals <= fit['window']))
                    FPB = icesat2_toolkit.fit.calc_first_photon_bias(temporal_residuals[ii],
                        n_pulses,n_pixels,mean_dead_time,5e-11,ITERATE=20)
                except:
                    pass
                else:
                    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_mean_corr'].data[j] = -0.5*FPB['mean']*c
                    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_mean_corr'].mask[j] = False
                    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_mean_corr_sigma'].data[j] = 0.5*FPB['mean_sigma']*c
                    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_mean_corr_sigma'].mask[j] = False
                    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr'].data[j] = -0.5*FPB['median']*c
                    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr'].mask[j] = False
                    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr_sigma'].data[j] = 0.5*FPB['median_sigma']*c
                    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr_sigma'].mask[j] = False
                    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_n_corr'].data[j] = np.copy(FPB['count'])
                    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_n_corr'].mask[j] = False
                    # first photon bias correction from CAL-19
                    FPB_calibrated = CAL19.ev(FPB['strength'],FPB['width'])
                    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_cal_corr'].data[j] = -0.5*FPB_calibrated*c
                    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_cal_corr'].mask[j] = False
                # estimate transmit pulse shape correction
                try:
                    W_RX = 2.0*fit['RDE']/c
                    dt_W = 2.0*fit['window']/c
                    TPS = icesat2_toolkit.fit.calc_transmit_pulse_shape(t_TX, p_TX, W_TX, W_RX,
                        dt_W, snr, ITERATE=50)
                except:
                    pass
                else:
                    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['tx_mean_corr'].data[j] = 0.5*TPS['mean']*c
                    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['tx_mean_corr'].mask[j] = False
                    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['tx_med_corr'].data[j] = 0.5*TPS['median']*c
                    IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['tx_med_corr'].mask[j] = False

        # if there is a valid land ice height
        if (~IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['h_mean'].mask[j]):
            # land ice height corrected for first photon bias and transmit-pulse shape
            # segment heights have already been "re-tided"
            IS2_atl03_fit[gtx]['land_ice_segments']['h_li'].data[j] = \
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['h_mean'].data[j] + \
                IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr'].data[j] + \
                IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['tx_med_corr'].data[j]
            IS2_atl03_fit[gtx]['land_ice_segments']['h_li'].mask[j] = False
            # land ice height errors (max of fit or first photon bias uncertainties)
            IS2_atl03_fit[gtx]['land_ice_segments']['h_li_sigma'].data[j] = np.sqrt(np.max([
                IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['sigma_h_mean'].data[j]**2,
                IS2_atl03_fit[gtx]['land_ice_segments']['bias_correction']['fpb_med_corr_sigma'].data[j]**2]))
            IS2_atl03_fit[gtx]['land_ice_segments']['h_li_sigma'].mask[j] = False

# calculate across-track slopes and geolocation uncertainties
# for each output beam
for gtx in sorted(IS2_atl03_beams):
    # data and attributes for beam gtx
    val = IS2_atl03_mds[gtx]
    attrs = IS2_atl03_attrs[gtx]
    # complementary beam in pair
    cmp = associated_beam_pair[gtx]
    # atmospheric profile for beam gtx from ATL09 dataset
    pfl = attrs['atmosphere_profile']
    # interpolate ATL03 segment-level variables using delta times
    valid, = np.nonzero(~IS2_atl03_fit[gtx]['land_ice_segments']['delta_time'].mask)

    # geolocation uncertainty
    sigma_geo_across = np.zeros((n_seg))
    sigma_geo_along = np.zeros((n_seg))
    sigma_geo_h = np.zeros((n_seg))
    # interpolate geolocation uncertainty from reference photon delta times
    S1 = scipy.interpolate.UnivariateSpline(val['geolocation']['delta_time'][:],
        val['geolocation']['sigma_across'][:],k=3,s=0)
    S2 = scipy.interpolate.UnivariateSpline(val['geolocation']['delta_time'][:],
        val['geolocation']['sigma_along'][:],k=3,s=0)
    S3 = scipy.interpolate.UnivariateSpline(val['geolocation']['delta_time'][:],
        val['geolocation']['sigma_h'][:],k=3,s=0)
    sigma_geo_across[valid] = S1(IS2_atl03_fit[gtx]['land_ice_segments']['delta_time'][valid])
    sigma_geo_along[valid] = S2(IS2_atl03_fit[gtx]['land_ice_segments']['delta_time'][valid])
    sigma_geo_h[valid] = S3(IS2_atl03_fit[gtx]['land_ice_segments']['delta_time'][valid])

    # extract and interpolate atmospheric parameters from ATL09
    dtime = IS2_atl03_fit[gtx]['land_ice_segments']['delta_time'].data[valid]
    IS2_atl09_mds,IS2_atl09_attrs = read_HDF5_ATL09(os.path.join(base_dir,ATL09_file),
        pfl, dtime, ATTRIBUTES=True)
    
    # segment fit across-track slopes
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dy'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dy'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dy'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dy'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dy']['units'] = "meters/meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dy']['contentType'] = "modelResult"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dy']['long_name'] = "Across Track Slope"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dy']['description'] = ("Across track slope "
        "from segment fits to weak and strong beam; the same slope is reported for both laser beams in each pair")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dy']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    # segment fit across-track slope errors
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dy_sigma'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dy_sigma'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dy_sigma'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dy_sigma'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dy_sigma']['units'] = "meters/meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dy_sigma']['contentType'] = "qualityInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dy_sigma']['long_name'] = "Sigma of Across Track Slope"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dy_sigma']['description'] = ("Propagated error in "
        "the across-track segment slope calculated from segment fits to weak and strong beam")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dy_sigma']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    # contribution of geolocation uncertainty to height error
    IS2_atl03_fit[gtx]['land_ice_segments']['sigma_geo_h'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['sigma_geo_h'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['sigma_geo_h'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['sigma_geo_h'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['sigma_geo_h']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['sigma_geo_h']['contentType'] = "qualityInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['sigma_geo_h']['long_name'] = "Vertical Geolocation Error"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['sigma_geo_h']['description'] = ("Total vertical geolocation error "
        "due to PPD and POD, including the effects of horizontal geolocation error on the segment vertical error.")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['sigma_geo_h']['coordinates'] = \
        "segment_id delta_time latitude longitude"
    
    # verify that complementary beam pair is in list of beams
    complement_count = n_seg if (cmp in IS2_atl03_beams) else 0
    # run for each geoseg (distributed over comm.size # of processes)
    for j in range(complement_count):
        # across track slopes for beam
        if ((~IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['h_mean'].mask[j]) & 
            (~IS2_atl03_fit[cmp]['land_ice_segments']['fit_statistics']['h_mean'].mask[j])):
            # segment fit across-track slopes
            dH = IS2_atl03_fit[gtx]['land_ice_segments']['h_li'].data[j] - \
                IS2_atl03_fit[cmp]['land_ice_segments']['h_li'].data[j]
            dY = IS2_atl03_fit[gtx]['land_ice_segments']['ground_track']['y_atc'].data[j] - \
                IS2_atl03_fit[cmp]['land_ice_segments']['ground_track']['y_atc'].data[j]
            IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dy'].data[j] = dH/dY
            IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dy'].mask[j] = False
            # segment fit across-track slope errors
            IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dy_sigma'].data[j] = np.sqrt(
                IS2_atl03_fit[gtx]['land_ice_segments']['h_li_sigma'].data[j]**2 +
                IS2_atl03_fit[cmp]['land_ice_segments']['h_li_sigma'].data[j]**2)/np.abs(dY)
            IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dy_sigma'].mask[j] = False
            # contribution of geolocation uncertainty to height errors
            IS2_atl03_fit[gtx]['land_ice_segments']['sigma_geo_h'].data[j] = np.sqrt(sigma_geo_h[j]**2 +
                (sigma_geo_along[j]*IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dx'].data[j])**2 +
                (sigma_geo_across[j]*IS2_atl03_fit[gtx]['land_ice_segments']['fit_statistics']['dh_fit_dy'].data[j])**2)
            IS2_atl03_fit[gtx]['land_ice_segments']['sigma_geo_h'].mask[j] = False
    
    # geoid height
    IS2_atl03_fit[gtx]['land_ice_segments']['dem']['geoid_h'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['dem']['geoid_h'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['dem']['geoid_h'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['dem']['geoid_h'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['dem']['geoid_h']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['dem']['geoid_h']['contentType'] = "referenceInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['dem']['geoid_h']['long_name'] = "Geoid Height"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['dem']['geoid_h']['description'] = ("Geoid height above "
        "WGS-84 reference ellipsoid (range -107 to 86m)")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['dem']['geoid_h']['source'] = "EGM2008"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['dem']['geoid_h']['valid_min'] = -107
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['dem']['geoid_h']['valid_max'] = 86
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['dem']['geoid_h']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    # interpolate geoid height from reference photon delta times
    SPL = scipy.interpolate.UnivariateSpline(val['geolocation']['delta_time'][:],
        val['geophys_corr']['geoid'][:],k=3,s=0)
    IS2_atl03_fit[gtx]['land_ice_segments']['dem']['geoid_h'].data[valid] = \
        SPL(IS2_atl03_fit[gtx]['land_ice_segments']['delta_time'][valid])
    IS2_atl03_fit[gtx]['land_ice_segments']['dem']['geoid_h'].mask = False
    
    # blowing snow PSC flag
    var = IS2_atl09_mds[pfl]['high_rate']['bsnow_psc']
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['bsnow_psc'] = np.ma.zeros((n_seg),fill_value=-1,dtype=var.dtype)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['bsnow_psc'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['geophysical']['bsnow_psc'] = -1
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_psc'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_psc']['units'] = "1"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_psc']['contentType'] = "modelResult"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_psc']['long_name'] = "Blowing snow PSC flag"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_psc']['description'] = ("Indicates the "
        "potential for polar stratospheric clouds to affect the blowing snow retrieval, where 0=none and 3=maximum. "
        "This flag is a function of month and hemisphere and is only applied poleward of 60 north and south")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_psc']['flag_meanings'] = ("none slight "
        "moderate maximum_bsnow_PSC_affected")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_psc']['flag_values'] = [0,1,2,3]
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_psc']['valid_min'] = 0
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_psc']['valid_max'] = 3
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_psc']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    # copy blowing snow PSC flag for valid points
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['bsnow_psc'].data[valid] = var[valid]
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['bsnow_psc'].mask[valid] = False
    
    # blowing snow confidence
    var = IS2_atl09_mds[pfl]['high_rate']['bsnow_conf']
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['bsnow_conf'] = np.ma.zeros((n_seg),fill_value=-4,dtype=var.dtype)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['bsnow_conf'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['geophysical']['bsnow_conf'] = -4
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_conf'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_conf']['units'] = "1"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_conf']['contentType'] = "modelResult"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_conf']['long_name'] = "Blowing snow confidence"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_conf']['description'] = ("Indicates the blowing snow "
        "confidence, where -3=surface not detected; 2=no surface wind;-1=no scattering layer found; 0=no top layer found; "
        "1=none-little; 2=weak; 3=moderate; 4=moderate-high; 5=high; 6=very high")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_conf']['flag_meanings'] = ("surface_not_detected "
        "no_surface_wind no_scattering_layer_found no_top_layer_found none_little weak moderate moderate_high high very_high")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_conf']['flag_values'] = [-3,-2,-1,0,1,2,3,4,5,6]
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_conf']['valid_min'] = -3
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_conf']['valid_max'] = 6
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_conf']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    # copy blowing snow confidence for valid points
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['bsnow_conf'].data[valid] = var[valid]
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['bsnow_conf'].mask[valid] = False
    
    # blowing snow optical depth
    var = IS2_atl09_mds[pfl]['high_rate']['bsnow_od']
    fv = IS2_atl09_attrs[pfl]['high_rate']['bsnow_od']['_FillValue']
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['bsnow_od'] = np.ma.zeros((n_seg),fill_value=fv,dtype=var.dtype)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['bsnow_od'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['geophysical']['bsnow_od'] = fv
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_od'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_od']['units'] = "1"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_od']['contentType'] = "modelResult"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_od']['long_name'] = "Blowing snow OD"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_od']['description'] = "Blowing snow layer optical depth"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['bsnow_od']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    # copy blowing snow optical depth for valid points
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['bsnow_od'].data[valid] = var[valid]
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['bsnow_od'].mask[valid] = False
    
    # cloud flag ASR
    var = IS2_atl09_mds[pfl]['high_rate']['cloud_flag_asr']
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['cloud_flag_asr'] = np.ma.zeros((n_seg),fill_value=-1,dtype=var.dtype)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['cloud_flag_asr'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['geophysical']['cloud_flag_asr'] = -1
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['cloud_flag_asr'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['cloud_flag_asr']['units'] = "1"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['cloud_flag_asr']['contentType'] = "modelResult"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['cloud_flag_asr']['long_name'] = "Cloud Flag ASR"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['cloud_flag_asr']['description'] = ("Indicates the Cloud "
        "flag probability from apparent surface reflectance, where 0=clear with high confidence; 1=clear with medium "
        "confidence; 2=clear with low confidence; 3=cloudy with low confidence; 4=cloudy with medium confidence; 5=cloudy "
        "with high confidence; 6=unknown")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['cloud_flag_asr']['flag_meanings'] = ("clear_with_high_confidence "
        "clear_with_medium_confidence clear_with_low_confidence cloudy_with_low_confidence cloudy_with_medium_confidence "
        "cloudy_with_high_confidence unknown")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['cloud_flag_asr']['flag_values'] = [0,1,2,3,4,5,6]
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['cloud_flag_asr']['valid_min'] = 0
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['cloud_flag_asr']['valid_max'] = 6
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['cloud_flag_asr']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    # copy blowing snow optical depth for valid points
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['cloud_flag_asr'].data[valid] = var[valid]
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['cloud_flag_asr'].mask[valid] = False
    
    # cloud flag atm
    var = IS2_atl09_mds[pfl]['high_rate']['cloud_flag_atm']
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['cloud_flag_atm'] = np.ma.zeros((n_seg),fill_value=-1,dtype=var.dtype)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['cloud_flag_atm'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['geophysical']['cloud_flag_atm'] = -1
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['cloud_flag_atm'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['cloud_flag_atm']['units'] = "1"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['cloud_flag_atm']['contentType'] = "modelResult"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['cloud_flag_atm']['long_name'] = "Cloud Flag Atm"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['cloud_flag_atm']['description'] = ("Number of layers found "
        "from the backscatter profile using the DDA layer finder")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['cloud_flag_atm']['flag_values'] = [0,1,2,3,4,5,6,7,8,9,10]
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['cloud_flag_atm']['valid_min'] = 0
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['cloud_flag_atm']['valid_max'] = 10
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['cloud_flag_atm']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    # copy blowing snow optical depth for valid points
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['cloud_flag_atm'].data[valid] = var[valid]
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['cloud_flag_atm'].mask[valid] = False
    
    # multiple scattering warning flag
    var = IS2_atl09_mds[pfl]['high_rate']['msw_flag']
    fv = IS2_atl09_attrs[pfl]['high_rate']['msw_flag']['_FillValue']
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['msw_flag'] = np.ma.zeros((n_seg),fill_value=fv,dtype=var.dtype)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['msw_flag'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['geophysical']['msw_flag'] = fv
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['msw_flag'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['msw_flag']['units'] = "1"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['msw_flag']['contentType'] = "referenceInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['msw_flag']['long_name'] = "Multiple Scattering Warning Flag"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['msw_flag']['description'] = ("Combined flag indicating the "
        "risks of severe multiple scattering. The multiple scattering warning flag (ATL09 parameter msw_flag) has values from "
        "-1 to 5 where zero means no multiple scattering and 5 the greatest. If no layers were detected, then msw_flag = 0. "
        "If blowing snow is detected and its estimated optical depth is greater than or equal to 0.5, then msw_flag = 5. "
        "If the blowing snow optical depth is less than 0.5, then msw_flag = 4. If no blowing snow is detected but there are "
        "cloud or aerosol layers detected, the msw_flag assumes values of 1 to 3 based on the height of the bottom of the "
        "lowest layer: < 1 km, msw_flag = 3; 1-3 km, msw_flag = 2; > 3km, msw_flag = 1. A value of -1 indicates that the "
        "signal to noise of the data was too low to reliably ascertain the presence of cloud or blowing snow. We expect "
        "values of -1 to occur only during daylight.")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['msw_flag']['flag_meanings'] = ("cannot_determine "
        "no_layers layer_gt_3km layer_between_1_and_3_km layer_lt_1km blow_snow_od_lt_0.5 blow_snow_od_gt_0.5")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['msw_flag']['flag_values'] = [-1,0,1,2,3,4,5]
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['msw_flag']['valid_min'] = -1
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['msw_flag']['valid_max'] = 5
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['msw_flag']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    # copy blowing snow optical depth for valid points
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['msw_flag'].data[valid] = var[valid]
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['msw_flag'].mask[valid] = False
    
    # range bias correction
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['range_bias_corr'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['range_bias_corr'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['geophysical']['range_bias_corr'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['range_bias_corr'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['range_bias_corr']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['range_bias_corr']['contentType'] = "referenceInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['range_bias_corr']['long_name'] = "Range bias correction"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['range_bias_corr']['description'] = "The range_bias estimated from geolocation analysis"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['range_bias_corr']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    # interpolate range bias correction from reference photon delta times
    SPL = scipy.interpolate.UnivariateSpline(val['geolocation']['delta_time'][:],
        val['geolocation']['range_bias_corr'][:],k=3,s=0)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['range_bias_corr'].data[valid] = \
        SPL(IS2_atl03_fit[gtx]['land_ice_segments']['delta_time'][valid])
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['range_bias_corr'].mask = False
    
    # total neutral atmosphere delay correction
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['neutat_delay_total'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['neutat_delay_total'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['geophysical']['neutat_delay_total'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['neutat_delay_total'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['neutat_delay_total']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['neutat_delay_total']['contentType'] = "referenceInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['neutat_delay_total']['long_name'] = "Total Neutral Atmospheric Delay"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['neutat_delay_total']['description'] = "Total neutral atmosphere delay correction (wet+dry)"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['neutat_delay_total']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    # interpolate neutral atmosphere delay from reference photon delta times
    SPL = scipy.interpolate.UnivariateSpline(val['geolocation']['delta_time'][:],
        val['geolocation']['neutat_delay_total'][:],k=3,s=0)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['neutat_delay_total'].data[valid] = \
        SPL(IS2_atl03_fit[gtx]['land_ice_segments']['delta_time'][valid])
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['neutat_delay_total'].mask = False
    
    # solar elevation
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['solar_elevation'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['solar_elevation'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['geophysical']['solar_elevation'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['solar_elevation'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['solar_elevation']['units'] = "degrees"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['solar_elevation']['contentType'] = "referenceInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['solar_elevation']['long_name'] = "Solar elevation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['solar_elevation']['description'] = ("Solar Angle above "
        "or below the plane tangent to the ellipsoid surface at the laser spot. Positive values mean the sun is above the "
        "horizon, while negative values mean it is below the horizon. The effect of atmospheric refraction is not included.")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['solar_elevation']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    # interpolate solar elevation from reference photon delta times
    SPL = scipy.interpolate.UnivariateSpline(val['geolocation']['delta_time'][:],
        val['geolocation']['solar_elevation'][:],k=3,s=0)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['solar_elevation'].data[valid] = \
        SPL(IS2_atl03_fit[gtx]['land_ice_segments']['delta_time'][valid])
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['solar_elevation'].mask = False
    
    # solar azimuth
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['solar_azimuth'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['solar_azimuth'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['geophysical']['solar_azimuth'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['solar_azimuth'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['solar_azimuth']['units'] = "degrees_east"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['solar_azimuth']['contentType'] = "referenceInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['solar_azimuth']['long_name'] = "Solar azimuth"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['solar_azimuth']['description'] = ("The direction, "
        "eastwards from north, of the sun vector as seen by an observer at the laser ground spot.")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['solar_azimuth']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    # interpolate solar azimuth from reference photon delta times
    SPL = scipy.interpolate.UnivariateSpline(val['geolocation']['delta_time'][:],
        val['geolocation']['solar_azimuth'][:],k=3,s=0)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['solar_azimuth'].data[valid] = \
        SPL(IS2_atl03_fit[gtx]['land_ice_segments']['delta_time'][valid])
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['solar_azimuth'].mask = False
    
    # geophysical correction values at segment reference photons
    # dynamic atmospheric correction
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['dac'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['dac'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['geophysical']['dac'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['dac'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['dac']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['dac']['contentType'] = "referenceInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['dac']['long_name'] = "Dynamic Atmosphere Correction"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['dac']['description'] = ("Dynamic Atmospheric Correction "
        "(DAC) includes inverted barometer (IB) effect")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['dac']['source'] = 'Mog2D-G'
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['dac']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    # interpolate dynamic atmospheric correction from reference photon delta times
    SPL = scipy.interpolate.UnivariateSpline(val['geolocation']['delta_time'][:],
        val['geophys_corr']['dac'][:],k=3,s=0)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['dac'].data[valid] = \
        SPL(IS2_atl03_fit[gtx]['land_ice_segments']['delta_time'][valid])
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['dac'].mask = False

    # solid earth tide
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['tide_earth'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['tide_earth'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['geophysical']['tide_earth'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_earth'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_earth']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_earth']['contentType'] = "referenceInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_earth']['long_name'] = "Earth Tide"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_earth']['description'] = "Solid Earth Tide"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_earth']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    # interpolate solid earth tide from reference photon delta times
    SPL = scipy.interpolate.UnivariateSpline(val['geolocation']['delta_time'][:],
        val['geophys_corr']['tide_earth'][:],k=3,s=0)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['tide_earth'].data[valid] = \
        SPL(IS2_atl03_fit[gtx]['land_ice_segments']['delta_time'][valid])
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['tide_earth'].mask = False
    
    # load tide
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['tide_load'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['tide_load'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['geophysical']['tide_load'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_load'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_load']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_load']['contentType'] = "referenceInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_load']['long_name'] = "Load Tide"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_load']['description'] = ("Load Tide - Local "
        "displacement due to Ocean Loading (-6 to 0 cm)")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_load']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    # interpolate load tide from reference photon delta times
    SPL = scipy.interpolate.UnivariateSpline(val['geolocation']['delta_time'][:],
        val['geophys_corr']['tide_load'][:],k=3,s=0)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['tide_load'].data[valid] = \
        SPL(IS2_atl03_fit[gtx]['land_ice_segments']['delta_time'][valid])
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['tide_load'].mask = False
    
    # ocean tide
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['tide_ocean'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['tide_ocean'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['geophysical']['tide_ocean'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_ocean'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_ocean']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_ocean']['contentType'] = "referenceInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_ocean']['long_name'] = "Ocean Tide"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_ocean']['description'] = ("Ocean Tides "
        "including diurnal and semi-diurnal (harmonic analysis), and longer period tides (dynamic and "
        "self-consistent equilibrium).")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_ocean']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    # interpolate ocean tide from reference photon delta times
    SPL = scipy.interpolate.UnivariateSpline(val['geolocation']['delta_time'][:],
        val['geophys_corr']['tide_ocean'][:],k=3,s=0)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['tide_ocean'].data[valid] = \
        SPL(IS2_atl03_fit[gtx]['land_ice_segments']['delta_time'][valid])
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['tide_ocean'].mask = False
    
    # ocean pole tide
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['tide_oc_pole'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['tide_oc_pole'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['geophysical']['tide_oc_pole'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_oc_pole'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_oc_pole']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_oc_pole']['contentType'] = "referenceInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_oc_pole']['long_name'] = "Ocean Pole Tide"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_oc_pole']['description'] = ("Oceanic surface "
        "rotational deformation due to polar motion (-2 to 2 mm).")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_oc_pole']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    # interpolate ocean pole tide from reference photon delta times
    SPL = scipy.interpolate.UnivariateSpline(val['geolocation']['delta_time'][:],
        val['geophys_corr']['tide_oc_pole'][:],k=3,s=0)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['tide_oc_pole'].data[valid] = \
        SPL(IS2_atl03_fit[gtx]['land_ice_segments']['delta_time'][valid])
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['tide_oc_pole'].mask = False
    
    # pole tide
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['tide_pole'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['tide_pole'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['geophysical']['tide_pole'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_pole'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_pole']['units'] = "meters"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_pole']['contentType'] = "referenceInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_pole']['long_name'] = "Solid Earth Pole Tide"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_pole']['description'] = ("Solid Earth Pole "
        "Tide - Rotational deformation due to polar motion  (-1.5 to 1.5 cm).")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['geophysical']['tide_pole']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    # interpolate pole tide from reference photon delta times
    SPL = scipy.interpolate.UnivariateSpline(val['geolocation']['delta_time'][:],
        val['geophys_corr']['tide_pole'][:],k=3,s=0)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['tide_pole'].data[valid] = \
        SPL(IS2_atl03_fit[gtx]['land_ice_segments']['delta_time'][valid])
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['tide_pole'].mask = False
    
    # elevation
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['ref_elev'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['ref_elev'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['geophysical']['ref_elev'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['ref_elev'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['ref_elev']['units'] = "radians"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['ref_elev']['contentType'] = "referenceInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['ref_elev']['long_name'] = "Elevation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['ref_elev']['description'] = ("Elevation of the "
        "unit pointing vector for the reference photon in the local ENU frame in radians.  The angle is measured "
        "from East-North plane and positive towards Up")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['ref_elev']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    # interpolate reference elevation from reference photon delta times
    SPL = scipy.interpolate.UnivariateSpline(val['geolocation']['delta_time'][:],
        val['geophys_corr']['ref_elev'][:],k=3,s=0)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['ref_elev'].data[valid] = \
        SPL(IS2_atl03_fit[gtx]['land_ice_segments']['delta_time'][valid])
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['ref_elev'].mask = False
    
    # azimuth
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['ref_azimuth'] = np.ma.zeros((n_seg),fill_value=fill_value)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['ref_azimuth'].mask = np.ones((n_seg),dtype=bool)
    IS2_atl03_fill[gtx]['land_ice_segments']['geophysical']['ref_azimuth'] = fill_value
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['ref_azimuth'] = {}
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['ref_azimuth']['units'] = "radians"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['ref_azimuth']['contentType'] = "referenceInformation"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['ref_azimuth']['long_name'] = "Azimuth"
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['ref_azimuth']['description'] = ("Azimuth of the "
        "unit pointing vector for the reference photon in the local ENU frame in radians.  The angle is measured "
        "from North and positive towards East")
    IS2_atl03_fit_attrs[gtx]['land_ice_segments']['ground_track']['ref_azimuth']['coordinates'] = \
        "../segment_id ../delta_time ../latitude ../longitude"
    # interpolate reference azimuth from reference photon delta times
    SPL = scipy.interpolate.UnivariateSpline(val['geolocation']['delta_time'][:],
        val['geophys_corr']['ref_azimuth'][:],k=3,s=0)
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['ref_azimuth'].data[valid] = \
        SPL(IS2_atl03_fit[gtx]['land_ice_segments']['delta_time'][valid])
    IS2_atl03_fit[gtx]['land_ice_segments']['geophysical']['ref_azimuth'].mask = False

#### Function for saving data to HDF5 file

In [ ]:
#-- PURPOSE: outputting the reduced and corrected ICESat-2 data to HDF5
def HDF5_ATL03_write(IS2_atl03_data, IS2_atl03_attrs, INPUT=None,
    FILENAME='', FILL_VALUE=None, CLOBBER=True):
    #-- setting HDF5 clobber attribute
    if CLOBBER:
        clobber = 'w'
    else:
        clobber = 'w-'

    #-- open output HDF5 file
    fileID = h5py.File(FILENAME, clobber)
    logging.info(FILENAME)

    #-- create HDF5 records
    h5 = {}

    # #-- ICESat-2 spacecraft orientation at time
    # fileID.create_group('orbit_info')
    # h5['orbit_info'] = {}
    # for k,v in IS2_atl03_data['orbit_info'].items():
    #     #-- Defining the HDF5 dataset variables
    #     val = 'orbit_info/{0}'.format(k)
    #     h5['orbit_info'][k] = fileID.create_dataset(val, np.shape(v), data=v,
    #         dtype=v.dtype, compression='gzip')
    #     #-- add HDF5 variable attributes
    #     for att_name,att_val in IS2_atl03_attrs['orbit_info'][k].items():
    #         h5['orbit_info'][k].attrs[att_name] = att_val

    #-- information ancillary to the data product
    #-- number of GPS seconds between the GPS epoch (1980-01-06T00:00:00Z UTC)
    #-- and ATLAS Standard Data Product (SDP) epoch (2018-01-01T00:00:00Z UTC)
    h5['ancillary_data'] = {}
    for k in ['atlas_sdp_gps_epoch','data_end_utc','data_start_utc','end_cycle',
        'end_geoseg','end_gpssow','end_gpsweek','end_orbit','end_region',
        'end_rgt','granule_end_utc','granule_start_utc','release','start_cycle',
        'start_geoseg','start_gpssow','start_gpsweek','start_orbit','start_region',
        'start_rgt','version']:
        #-- Defining the HDF5 dataset variables
        v = IS2_atl03_data['ancillary_data'][k]
        val = 'ancillary_data/{0}'.format(k)
        h5['ancillary_data'][k] = fileID.create_dataset(val, np.shape(v), data=v,
            dtype=v.dtype)
        #-- add HDF5 variable attributes
        for att_name,att_val in IS2_atl03_attrs['ancillary_data'][k].items():
            h5['ancillary_data'][k].attrs[att_name] = att_val

    #-- land_ice_segments variable groups for each beam
    GROUPS=['fit_statistics','geophysical','ground_track','dem','bias_correction']
    #-- write each output beam
    for gtx in ['gt1l','gt1r','gt2l','gt2r','gt3l','gt3r']:
        fileID.create_group(gtx)
        fileID['ancillary_data'].create_group(gtx)

        #-- add HDF5 group attributes for beam
        for att_name in ['Description','atlas_pce','atlas_beam_type',
            'groundtrack_id','atmosphere_profile','atlas_spot_number',
            'sc_orientation']:
            fileID[gtx].attrs[att_name] = IS2_atl03_attrs[gtx][att_name]

        #-- add transmit pulse shape and dead time parameters
        h5['ancillary_data'][gtx] = {}
        for k,v in IS2_atl03_data['ancillary_data'][gtx].items():
            #-- attributes
            attrs = IS2_atl03_attrs['ancillary_data'][gtx][k]
            #-- Defining the HDF5 dataset variables
            val = 'ancillary_data/{0}/{1}'.format(gtx,k)
            h5['ancillary_data'][gtx][k] = fileID.create_dataset(val,
                np.shape(v), data=v, dtype=v.dtype)
            #-- add HDF5 variable attributes
            for att_name,att_val in attrs.items():
                h5['ancillary_data'][gtx][k].attrs[att_name] = att_val

        #-- create land_ice_segments group
        fileID[gtx].create_group('land_ice_segments')
        h5[gtx] = dict(land_ice_segments={})
        for att_name in ['Description','data_rate']:
            att_val = IS2_atl03_attrs[gtx]['land_ice_segments'][att_name]
            fileID[gtx]['land_ice_segments'].attrs[att_name] = att_val

        #-- segment_id
        v = IS2_atl03_data[gtx]['land_ice_segments']['segment_id']
        attrs = IS2_atl03_attrs[gtx]['land_ice_segments']['segment_id']
        #-- Defining the HDF5 dataset variables
        val = '{0}/{1}/{2}'.format(gtx,'land_ice_segments','segment_id')
        h5[gtx]['land_ice_segments']['segment_id'] = fileID.create_dataset(val,
            np.shape(v), data=v, dtype=v.dtype, compression='gzip')
        #-- add HDF5 variable attributes
        for att_name,att_val in attrs.items():
            h5[gtx]['land_ice_segments']['segment_id'].attrs[att_name] = att_val

        #-- geolocation, time and height variables
        for k in ['latitude','longitude','delta_time','h_li','h_li_sigma',
            'sigma_geo_h','atl06_quality_summary']:
            #-- values and attributes
            v = IS2_atl03_data[gtx]['land_ice_segments'][k]
            attrs = IS2_atl03_attrs[gtx]['land_ice_segments'][k]
            fillvalue = FILL_VALUE[gtx]['land_ice_segments'][k]
            #-- Defining the HDF5 dataset variables
            val = '{0}/{1}/{2}'.format(gtx,'land_ice_segments',k)
            h5[gtx]['land_ice_segments'][k] = fileID.create_dataset(val,
                np.shape(v), data=v, dtype=v.dtype, fillvalue=fillvalue,
                compression='gzip')
            #-- attach dimensions
            for dim in ['segment_id']:
                h5[gtx]['land_ice_segments'][k].dims.create_scale(
                    h5[gtx]['land_ice_segments'][dim], dim)
                h5[gtx]['land_ice_segments'][k].dims[0].attach_scale(
                    h5[gtx]['land_ice_segments'][dim])
            #-- add HDF5 variable attributes
            for att_name,att_val in attrs.items():
                h5[gtx]['land_ice_segments'][k].attrs[att_name] = att_val

        #-- fit statistics, geophysical corrections, geolocation and dem
        for key in GROUPS:
            fileID[gtx]['land_ice_segments'].create_group(key)
            h5[gtx]['land_ice_segments'][key] = {}
            for att_name in ['Description','data_rate']:
                att_val=IS2_atl03_attrs[gtx]['land_ice_segments'][key][att_name]
                fileID[gtx]['land_ice_segments'][key].attrs[att_name] = att_val
            for k,v in IS2_atl03_data[gtx]['land_ice_segments'][key].items():
                #-- attributes
                attrs = IS2_atl03_attrs[gtx]['land_ice_segments'][key][k]
                fillvalue = FILL_VALUE[gtx]['land_ice_segments'][key][k]
                #-- Defining the HDF5 dataset variables
                val = '{0}/{1}/{2}/{3}'.format(gtx,'land_ice_segments',key,k)
                if fillvalue:
                    h5[gtx]['land_ice_segments'][key][k] = \
                        fileID.create_dataset(val, np.shape(v), data=v,
                            dtype=v.dtype, fillvalue=fillvalue, compression='gzip')
                else:
                    h5[gtx]['land_ice_segments'][key][k] = \
                        fileID.create_dataset(val, np.shape(v), data=v,
                            dtype=v.dtype, compression='gzip')
                #-- attach dimensions
                for dim in ['segment_id']:
                    h5[gtx]['land_ice_segments'][key][k].dims.create_scale(
                        h5[gtx]['land_ice_segments'][dim], dim)
                    h5[gtx]['land_ice_segments'][key][k].dims[0].attach_scale(
                        h5[gtx]['land_ice_segments'][dim])
                #-- add HDF5 variable attributes
                for att_name,att_val in attrs.items():
                    h5[gtx]['land_ice_segments'][key][k].attrs[att_name] = att_val

    #-- HDF5 file title
    fileID.attrs['featureType'] = 'trajectory'
    fileID.attrs['title'] = 'ATLAS/ICESat-2 Land Ice Height'
    fileID.attrs['summary'] = ('Estimates of the ice-sheet mean surface height '
        'relative to the WGS-84 ellipsoid, and ancillary parameters needed to '
        'interpret and assess the quality of these height estimates.')
    fileID.attrs['description'] = ('Land ice surface heights for each beam, '
        'along and across-track slopes calculated for beam pairs.  All '
        'parameters are calculated for the same along-track increments for '
        'each beam and repeat.')
    date_created = datetime.datetime.today()
    fileID.attrs['date_created'] = date_created.isoformat()
    project = 'ICESat-2 > Ice, Cloud, and land Elevation Satellite-2'
    fileID.attrs['project'] = project
    platform = 'ICESat-2 > Ice, Cloud, and land Elevation Satellite-2'
    fileID.attrs['project'] = platform
    #-- add attribute for elevation instrument and designated processing level
    instrument = 'ATLAS > Advanced Topographic Laser Altimeter System'
    fileID.attrs['instrument'] = instrument
    fileID.attrs['source'] = 'Spacecraft'
    fileID.attrs['references'] = 'https://nsidc.org/data/icesat-2'
    fileID.attrs['processing_level'] = '4'
    #-- add attributes for input ATL03 and ATL09 files
    fileID.attrs['input_files'] = ','.join([os.path.basename(i) for i in INPUT])
    #-- find geospatial and temporal ranges
    lnmn,lnmx,ltmn,ltmx,tmn,tmx = (np.inf,-np.inf,np.inf,-np.inf,np.inf,-np.inf)
    for gtx in ['gt1l','gt1r','gt2l','gt2r','gt3l','gt3r']:
        lon = IS2_atl03_data[gtx]['land_ice_segments']['longitude']
        lat = IS2_atl03_data[gtx]['land_ice_segments']['latitude']
        delta_time = IS2_atl03_data[gtx]['land_ice_segments']['delta_time']
        #-- setting the geospatial and temporal ranges
        lnmn = lon.min() if (lon.min() < lnmn) else lnmn
        lnmx = lon.max() if (lon.max() > lnmx) else lnmx
        ltmn = lat.min() if (lat.min() < ltmn) else ltmn
        ltmx = lat.max() if (lat.max() > ltmx) else ltmx
        tmn = delta_time.min() if (delta_time.min() < tmn) else tmn
        tmx = delta_time.max() if (delta_time.max() > tmx) else tmx
    #-- add geospatial and temporal attributes
    fileID.attrs['geospatial_lat_min'] = ltmn
    fileID.attrs['geospatial_lat_max'] = ltmx
    fileID.attrs['geospatial_lon_min'] = lnmn
    fileID.attrs['geospatial_lon_max'] = lnmx
    fileID.attrs['geospatial_lat_units'] = "degrees_north"
    fileID.attrs['geospatial_lon_units'] = "degrees_east"
    fileID.attrs['geospatial_ellipsoid'] = "WGS84"
    fileID.attrs['date_type'] = 'UTC'
    fileID.attrs['time_type'] = 'CCSDS UTC-A'
    #-- convert start and end time from ATLAS SDP seconds into UTC time
    time_utc = convert_delta_time(np.array([tmn,tmx]))
    #-- convert to calendar date
    YY,MM,DD,HH,MN,SS = icesat2_toolkit.time.convert_julian(time_utc['julian'],
        FORMAT='tuple')
    #-- add attributes with measurement date start, end and duration
    tcs = datetime.datetime(int(YY[0]), int(MM[0]), int(DD[0]),
        int(HH[0]), int(MN[0]), int(SS[0]), int(1e6*(SS[0] % 1)))
    fileID.attrs['time_coverage_start'] = tcs.isoformat()
    tce = datetime.datetime(int(YY[1]), int(MM[1]), int(DD[1]),
        int(HH[1]), int(MN[1]), int(SS[1]), int(1e6*(SS[1] % 1)))
    fileID.attrs['time_coverage_end'] = tce.isoformat()
    fileID.attrs['time_coverage_duration'] = '{0:0.0f}'.format(tmx-tmn)  
    #-- Closing the HDF5 file
    fileID.close()

#### Save to HDF5 File

In [ ]:
#-- use default output file name and path
ATL03_dir = os.path.dirname(ATL03_file)
args = (SUB,'ATL86',YY,MM,DD,HH,MN,SS,TRK,CYCL,GRAN,RL,VERS,AUX)
file_format='{0}{1}_{2}{3}{4}{5}{6}{7}_{8}{9}{10}_{11}_{12}{13}.h5'
output_file=os.path.join(ATL03_dir,file_format.format(*args))
#-- write to HDF5 file
HDF5_ATL03_write(IS2_atl03_fit, IS2_atl03_attrs,
    FILENAME=output_file,
    INPUT=[ATL03_file,ATL09_file],
    FILL_VALUE=IS2_atl03_fill,
    CLOBBER=True)